Il metodo scelto per l'assignment 1 è stato il metodo di Zhang [(Link alla pagina wikipedia)](https://en.wikipedia.org/wiki/Camera_resectioning#:~:text=Zhang%27s%20method%5Bedit%5D) 

I passaggi da effettuare sono
- Corrispondenza punti 2D e 3D
- Trovare la matrice P
- Usare fattorizzazione RQ (Variante della fattorizzazione QR)
- Trovare K che è la matrice R della fattorizzazione precedente
- Mentre la matrice R di rotazione (da non confondere con la R di RQ) sarà la matrice Q di RQ
- Trovare t vettore colonna della traslazione

Import di tutte le librerie necessarie
Usiamo numpy per la gestione delle matrici generate dalle immagini e math per fare i calcoli, glob ci aiuta a raccogliere tutti i file di un tipo 

In [1]:
import numpy as np
import math
import cv2
import glob
import os


Definiamo la dimensioni della scacchiera e i criteri per fermare l'iterazione della rifinitura dei contorni e dei punti interni della scacchiera

In [19]:
scacchiera = (7,9)
criteri = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

Definiamo quindi degli array per i punti nello spazio e nel piano ed inizializziamo i punti nello spazio a 0, 

In [ ]:
punti3d  = []
punti2d = [] 
oggetto3d = np.zeros((1, scacchiera[0] * scacchiera[1], 3), np.float32)
oggetto3d[0,:,:2] = np.mgrid[0:scacchiera[0], 0:scacchiera[1]].T.reshape(-1, 2)
prev_img_shape = None

errori = []

Raccogliamo tutte le immagini di tipo jpg presenti nella directory immagini

In [4]:
immagini = glob.glob('/immagini/*.jpg')

Creo la funzione custom calibrate camera

Definisco un doppio ciclo for, il primo serve per ciclare in un range 0,3 estremi inclusi per dividere il set di immagini in 4 per fare cross validation, il set di validazione sarà dato dal vettore tagliato dalla posizione iesima alla i+1 esima



In [ ]:
for i in range (0,4):
    validation = immagini[i,i+1]
    training = [i for i in immagini if i not in validation]
    for im in training:
        image = cv2.imread(im)
        scalagrigi = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

        ret, angoli = cv2.findChessboardCorners(scalagrigi, scacchiera, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

        if ret == True:
            punti3d.append(oggetto3d)
            angoli2d = cv2.cornerSubPix(scalagrigi, angoli, (11,11),(-1,-1), criteria)
            punti2d.append(angoli2d)

            immagine = cv2.drawChessboardCorners(im, scacchiera, angoli2d, ret)

        cv2.imshow('immagine', im)
        cv2.waitkey(0)
    cv2.destroyAllWindows()
    #errori[i] = cv2.calibrateCamera() - calibra()
 